In [23]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
#pinecone is an embedding data there are opensource options on the market 
from pinecone import Pinecone, ServerlessSpec


In [24]:
pc = Pinecone(api_key=os.getenv("PINCONE_API_KEY"))
#make sure dimension of pincone index match the model your using (update in future). 1536=openai
#metric is used for similairty search( there are better depending on use case)
pc.create_index(
    name="rag", dimension= 1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '5a53146d298e622cf0f90f734ad16a1b', 'Date': 'Wed, 21 Aug 2024 21:43:42 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [25]:
import json 
data= json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Engaging lectures and fair grading. Could improve on office hour availability.'},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Excellent teacher! Makes complex topics easy to understand. Highly recommended.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Interesting content, but lectures can be disorganized. Assignments are challenging.'},
 {'professor': 'Prof. Robert Anderson',
  'subject': 'History',
  'stars': 4,
  'review': 'Passionate about the subject. Assignments are thought-provoking. Sometimes talks too fast.'},
 {'professor': 'Dr. Lisa Patel',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Clear explanations and helpful lab sessions. Always available for questions.'},
 {'professor': 'Prof. David Wilson',
  'subject': 'Mathematics',
  'stars': 2,
  'review': 'Difficult to follow in class. 

In [26]:
processed_data=[]
client=OpenAI()
#code is different if use other models
for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        #specific to openai
        model="text-embedding-3-small",
    )
    embedding =response.data[0].embedding
    #make sure the data is in a correct format if your going to use other embbedding databases
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })


APIConnectionError: Connection error.